In [1]:
install.packages("ggplot2")
install.packages("dplyr")
install.packages("caret")
install.packages("glmnet")


The downloaded binary packages are in
	/var/folders/91/4c3th19d31j6hvx23r6bb5b00000gn/T//RtmpsQrJEh/downloaded_packages

The downloaded binary packages are in
	/var/folders/91/4c3th19d31j6hvx23r6bb5b00000gn/T//RtmpsQrJEh/downloaded_packages

The downloaded binary packages are in
	/var/folders/91/4c3th19d31j6hvx23r6bb5b00000gn/T//RtmpsQrJEh/downloaded_packages

The downloaded binary packages are in
	/var/folders/91/4c3th19d31j6hvx23r6bb5b00000gn/T//RtmpsQrJEh/downloaded_packages


In [2]:
library(dplyr)
library(glmnet)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: Matrix

Loaded glmnet 4.1-8



# Import and format data

In [3]:
#read in the data
data <- read.csv("/Users/User/Documents/GitHub/DesktopDownthrottlingExperiment/post_experiment/Regression_2_(web_to_serviceable)/regression_tables/fieldwork_regression_table.csv")
data$scaled_score <- data$score_min/5
data$scaled_score <- pmax(pmin(data$scaled_score, 0.9999), 0.0001)
data$logistic_transformed_score <- log(data$scaled_score / (1 - data$scaled_score))
trim_data <- data %>% select(-identifier,-score_min, - scaled_score,  -serviceable_agg, -score_ellen, -score_aadya, -serviceable_ellen, -serviceable_aadya)

# Build a model

In [4]:
model <- lm(logistic_transformed_score ~ ., data = trim_data)
summary(model)


Call:
lm(formula = logistic_transformed_score ~ ., data = trim_data)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.95765 -0.29883 -0.00502  0.22484  1.42456 

Coefficients: (3 not defined because of singularities)
                                                     Estimate Std. Error
(Intercept)                                         2.345e+00  2.592e+00
IT01V_packetsRecieved_ellen                        -1.626e-03  6.398e-02
IT01V_packetsRecieved_aadya                         7.237e-02  6.080e-02
IT01V_packetsLost_ellen                            -1.201e-02  5.790e-02
IT01V_packetsLost_aadya                             2.545e-03  2.947e-02
IT01V_frameWidth_ellen                              1.712e-03  5.649e-03
IT01V_frameWidth_aadya                             -2.546e-04  1.163e-03
IT01V_frameHeight_ellen                                    NA         NA
IT01V_frameHeight_aadya                                    NA         NA
IT01V_totalFreezesDuration_ellen        

# Penalised regression

## Lasso/ Ridge

In [5]:
X <- model.matrix(logistic_transformed_score ~ ., data = trim_data)[,-1]
y <- trim_data$logistic_transformed_score

# Fit the lasso model (alpha = 1 for Lasso)
lasso_model <- glmnet(X, y, alpha = 1)

# Predict on training data
lasso_predictions <- predict(lasso_model, X)

# Calculate MSE for lasso model
mse_lasso <- mean((y - lasso_predictions)^2)

In [6]:
# Fit the ridge model (alpha = 0)
ridge_model <- glmnet(X, y, alpha = 0)

# Predict on training data
ridge_predictions <- predict(ridge_model, X)

# Calculate MSE for lasso model
mse_ridge <- mean((y - ridge_predictions)^2)

## AIC

In [7]:
fit_aic <- step(model)
summary(fit_aic)

aic_predictions <- predict(fit_aic, trim_data)
# par(bg = "white")
# options(repr.plot.width = 10, repr.plot.height = 6)
# plot(predictions, trim_data$logistic_transformed_score, 
#      main = "Prediction performance of AIC stepfit model", 
#      xlab = "Predicted values", ylab = "Recorded values")

# Calculate Mean Squared Error (MSE)
mse_aic <- mean((trim_data$logistic_transformed_score - aic_predictions)^2)

Start:  AIC=-46.6
logistic_transformed_score ~ IT01V_packetsRecieved_ellen + IT01V_packetsRecieved_aadya + 
    IT01V_packetsLost_ellen + IT01V_packetsLost_aadya + IT01V_frameWidth_ellen + 
    IT01V_frameWidth_aadya + IT01V_frameHeight_ellen + IT01V_frameHeight_aadya + 
    IT01V_totalFreezesDuration_ellen + IT01V_totalFreezesDuration_aadya + 
    IT01V_framesPerSecond_ellen + IT01V_framesPerSecond_aadya + 
    IT01V_bytesReceived_in_bits_s_ellen + IT01V_bytesReceived_in_bits_s_aadya + 
    IT01V_totalProcessingDelay_ellen + IT01V_totalProcessingDelay_aadya + 
    IT01V_jitter_ellen + IT01V_jitter_aadya + IT01V_jitterBufferDelay_emissions_ellen + 
    IT01V_jitterBufferDelay_emissions_aadya + IT01A_bytesReceived_in_bits_s_ellen + 
    IT01A_bytesReceived_in_bits_s_aadya + IT01A_jitterBufferDelay_emissions_ellen + 
    IT01A_jitterBufferDelay_emissions_aadya + OT01V_packetsSent_s_ellen + 
    OT01V_packetsSent_s_aadya + OT01V_bytesSent_in_bits_s_ellen + 
    OT01V_bytesSent_in_bits_s_a


Call:
lm(formula = logistic_transformed_score ~ IT01V_packetsRecieved_aadya + 
    IT01V_totalFreezesDuration_ellen + IT01V_totalFreezesDuration_aadya + 
    IT01V_framesPerSecond_ellen + IT01V_framesPerSecond_aadya + 
    IT01V_bytesReceived_in_bits_s_aadya + IT01V_totalProcessingDelay_ellen + 
    IT01V_jitterBufferDelay_emissions_ellen + IT01A_jitterBufferDelay_emissions_ellen + 
    OT01V_packetsSent_s_aadya + OT01V_bytesSent_in_bits_s_aadya + 
    OT01V_framesPerSecond_ellen + OT01V_totalPacketSendDelay_aadya + 
    OT01V_totalPacketSendDelay_packetsSent_in_ms_ellen + OT01V_totalPacketSendDelay_packetsSent_in_ms_aadya + 
    RIV_roundTripTime_ellen + RIV_roundTripTime_aadya + RIA_fractionLost_aadya + 
    RIA_roundTripTime_aadya + AP_totalPlayoutDelay_ellen, data = trim_data)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.93527 -0.32707 -0.01506  0.31279  1.56469 

Coefficients:
                                                     Estimate Std. Error
(Intercept)    

## BIC

In [8]:
fit_bic<- step(model, k = log(length(trim_data$logistic_transformed_score)))
#summary(fit_aic)

bic_predictions <- predict(fit_bic, trim_data)
# par(bg = "white")
# options(repr.plot.width = 10, repr.plot.height = 6)
# plot(predictions, trim_data$logistic_transformed_score, 
#      main = "Prediction performance of AIC stepfit model", 
#      xlab = "Predicted values", ylab = "Recorded values")

# Calculate Mean Squared Error (MSE)
mse_bic <- mean((trim_data$logistic_transformed_score - bic_predictions)^2)

Start:  AIC=68.39
logistic_transformed_score ~ IT01V_packetsRecieved_ellen + IT01V_packetsRecieved_aadya + 
    IT01V_packetsLost_ellen + IT01V_packetsLost_aadya + IT01V_frameWidth_ellen + 
    IT01V_frameWidth_aadya + IT01V_frameHeight_ellen + IT01V_frameHeight_aadya + 
    IT01V_totalFreezesDuration_ellen + IT01V_totalFreezesDuration_aadya + 
    IT01V_framesPerSecond_ellen + IT01V_framesPerSecond_aadya + 
    IT01V_bytesReceived_in_bits_s_ellen + IT01V_bytesReceived_in_bits_s_aadya + 
    IT01V_totalProcessingDelay_ellen + IT01V_totalProcessingDelay_aadya + 
    IT01V_jitter_ellen + IT01V_jitter_aadya + IT01V_jitterBufferDelay_emissions_ellen + 
    IT01V_jitterBufferDelay_emissions_aadya + IT01A_bytesReceived_in_bits_s_ellen + 
    IT01A_bytesReceived_in_bits_s_aadya + IT01A_jitterBufferDelay_emissions_ellen + 
    IT01A_jitterBufferDelay_emissions_aadya + OT01V_packetsSent_s_ellen + 
    OT01V_packetsSent_s_aadya + OT01V_bytesSent_in_bits_s_ellen + 
    OT01V_bytesSent_in_bits_s_a

# Evaluate all the models

In [25]:
cat("MSE for AIC:", mse_aic, "\n")
cat("MSE for BIC:", mse_bic, "\n")
cat("MSE for lasso:", mse_lasso, "\n")
cat("MSE for ridge:", mse_ridge, "\n")

MSE for AIC: 0.2303441 
MSE for BIC: 0.2751526 
MSE for lasso: 0.4237797 
MSE for ridge: 0.9162658 
